In [1]:
import ee
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
load_dotenv()

ee.Authenticate()
projectName = os.getenv("PROJECT")
ee.Initialize(project=projectName)

In [2]:
i_date = '2022-01-01'
f_date = '2022-01-10'
scale = 55500
collection_name = 'NOAA/CPC/Precipitation'
band_name = 'precipitation'

# Load MODIS LST dataset
lst = ee.ImageCollection(collection_name).select(band_name).filterDate(i_date, f_date)
geometry = ee.Geometry.Polygon([
        [
            [
                120.74772644100017,
                17.15755081300017
            ],
            [
                120.68620300300017,
                17.166650772000025
            ],
            [
                120.66735077000021,
                17.193929673000127
            ],
            [
                120.68167114300002,
                17.198799133000136
            ],
            [
                120.68264770500002,
                17.255729676000158
            ],
            [
                120.61428833100001,
                17.310699462000173
            ],
            [
                120.55795288100012,
                17.307779313000083
            ],
            [
                120.53884124800004,
                17.357610703000034
            ],
            [
                120.57779693600003,
                17.42794990600015
            ],
            [
                120.58556366000016,
                17.476249695000092
            ],
            [
                120.52526092500023,
                17.49980926600017
            ],
            [
                120.47895050000011,
                17.481849670000088
            ],
            [
                120.46806335400015,
                17.514780045000123
            ],
            [
                120.48757171600005,
                17.545000076000008
            ],
            [
                120.5045928960003,
                17.64545059300002
            ],
            [
                120.58183288500015,
                17.799249650000036
            ],
            [
                120.69017028700011,
                17.834899903000064
            ],
            [
                120.72754669100004,
                17.892890931000125
            ],
            [
                120.78321838400018,
                17.911699295000062
            ],
            [
                120.8332366940001,
                17.959980012000074
            ],
            [
                120.9097366330002,
                17.950580597000112
            ],
            [
                120.92783355600011,
                17.978069305000133
            ],
            [
                120.98722839400023,
                17.939619064999988
            ],
            [
                121.0101699820002,
                17.843719482000154
            ],
            [
                121.06674957300004,
                17.848140716000046
            ],
            [
                121.09979248100012,
                17.824209212000028
            ],
            [
                121.12270355300006,
                17.683950424999978
            ],
            [
                121.09150695800008,
                17.666749954000124
            ],
            [
                121.11972045800019,
                17.58934974700003
            ],
            [
                121.11019134600008,
                17.56086921700006
            ],
            [
                120.99800872700018,
                17.49139022900016
            ],
            [
                120.99442291200023,
                17.40873909100003
            ],
            [
                120.89766693000013,
                17.190799714000036
            ],
            [
                120.79891204800015,
                17.187879562000035
            ],
            [
                120.74772644100017,
                17.15755081300017
            ]
        ]
    ])


In [3]:
def reduce_image(image, geometry):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mean = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=scale,
        bestEffort=True
    ).get(band_name)
    return ee.Feature(None, {'date': date, band_name: mean})


In [4]:
province="Abra"

In [5]:
time_series = lst.map(lambda img: reduce_image(img, geometry=geometry)).filter(ee.Filter.notNull([band_name]))
time_series

In [6]:
df = pd.DataFrame(time_series.getInfo()['features'])
df

,type,geometry,id,properties
0,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220101_RT,"{'date': '2022-01-01', 'precipitation': 0}"
1,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220102_RT,"{'date': '2022-01-02', 'precipitation': 0.7700..."
2,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220103_RT,"{'date': '2022-01-03', 'precipitation': 0}"
3,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220104_RT,"{'date': '2022-01-04', 'precipitation': 0.0151..."
4,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220105_RT,"{'date': '2022-01-05', 'precipitation': 0.0378..."
5,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220106_RT,"{'date': '2022-01-06', 'precipitation': 0.0097..."
6,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220107_RT,"{'date': '2022-01-07', 'precipitation': 0}"
7,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220108_RT,"{'date': '2022-01-08', 'precipitation': 0.0240..."
8,Feature,None,PRCP_CU_GAUGE_V1_0GLB_0_50deg_lnx_20220109_RT,"{'date': '2022-01-09', 'precipitation': 0.1204..."


In [7]:
df = pd.json_normalize(df['properties'])
df

,date,precipitation
0,2022-01-01,0.000000
1,2022-01-02,0.770057
2,2022-01-03,0.000000
3,2022-01-04,0.015104
4,2022-01-05,0.037821
5,2022-01-06,0.009786
6,2022-01-07,0.000000
7,2022-01-08,0.024064
8,2022-01-09,0.120429


In [8]:
df['precipitation'] = df['precipitation']  + 1
df

,date,precipitation
0,2022-01-01,1.000000
1,2022-01-02,1.770057
2,2022-01-03,1.000000
3,2022-01-04,1.015104
4,2022-01-05,1.037821
5,2022-01-06,1.009786
6,2022-01-07,1.000000
7,2022-01-08,1.024064
8,2022-01-09,1.120429


In [7]:
import pandas as pd

# Sample DataFrame with 3 columns
data = {
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50],
    'C': [100, 200, 300, 400, 500]
}
df = pd.DataFrame(data)

data2 = {
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50],
    'C': [100, 200, 300, 400, 500]
}
df2 = pd.DataFrame(data2)

# Extend by repeating the pattern
df_extended = pd.concat([df, df2], ignore_index=True)  # Extends to 3 times its length

df_extended


,A,B,C
0,1,10,100
1,2,20,200
2,3,30,300
3,4,40,400
4,5,50,500
5,1,10,100
6,2,20,200
7,3,30,300
8,4,40,400
9,5,50,500
